<a href="https://colab.research.google.com/github/speedsters/Adaboost_cancer_prediction/blob/main/Spacy_Model_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Create spacy model
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6 MB 4.3 MB/s 
     |████████████████████████████████| 819 kB 32.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.2.0
    Uninstalling setuptools-57.2.0:
      Successfully uninstalled setuptools-57.2.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 6.4 MB 4.1 MB/s 
     |████████████████████████████████| 10.1 MB 46.3 MB/s 
     |████████████████████████████████| 456 kB 43.1 MB/s 
     |████████████████████████████████| 621 kB 45.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  distutils: /usr/local/include/python3.7/catalogue
  sysconfig: /usr/include/python3.7m/catalogue
  distutils: /usr/local/include/pyt

In [2]:
#Load Libraries
import os
import os.path
from os import path
import csv
import nltk
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import plac
import random
import spacy

In [3]:
import json
import spacy
from spacy.tokens import Doc
from spacy.training import Example

In [4]:
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [14]:
#datafortraining=pd.read_csv('outputall.csv')
excel_file=pd.ExcelFile('outputall.xlsx')
datafortraining=excel_file.parse(excel_file.sheet_names[0])

In [15]:
datafortraining.drop(labels=['Unmatched'], axis=1,inplace = True)

KeyError: ignored

In [16]:
datafortraining

,text,subtext,entity
0,_ pdt occupancy sensors & photocells,sensor,control type
1,_ pdt occupancy sensor & photocell,sensor,control type
2,# 4ft emergency sections w/ battery pack 1250...,battery,power source
3,# 4ft emergency sections w/ battery pack 1250...,battery pack,accessory type
4,# 4ft emergency sections w/ batteries packs 1...,battery,power source
...,...,...,...
8445,zigbee radio module required. enter quantity. ...,zigbee,wireless technology
8446,zigbee radio module required. enter quantity. ...,control,product type
8447,zigbee radio module required. enter quantity. ...,zigbee,wireless technology
8448,zigbee radio module required. enter quantity.*...,control,product type


In [17]:
TRAIN_DATA = [
(".125 clear polycarbonate environmental side", {"entities": [(25, 29, "Product Type")]}),
(".125 Clear Polycarbonate Lamp Side", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]#excelcy = ExcelCy.execute(file_path='D:/source/repos/AttributeBasedDictionary/backend/outputall.xlsx')

In [48]:
import re
TRAIN_DATA = []
ent_list = []
for index, row in datafortraining.iterrows():
    # print(row['text'], row['subtext'], row['entity'])
    match=(re.search(row['subtext'], row['text']))
    ent_dic = {}
    if match:
    ## Fill value to the dictionary
        start = match.start()
        end = match.end()
        entity = row['entity']
        ent_list_sen = []
        
        ent_list_sen.append((start, end ,entity))
        ent_list.append(ent_list_sen)
         ## Create blank dictionary
        ent_dic = {}
        ## Fill value to the dictionary
        ent_dic["entities"] = ent_list[0]
        ## Prepare final training data
        TRAIN_DATA.append((row['text'],ent_dic))

In [49]:
TRAIN_DATA = TRAIN_DATA[:10]
TRAIN_DATA

[('_ pdt occupancy sensors & photocells',
  {'entities': [(16, 22, 'control type')]}),
 ('_ pdt occupancy sensor & photocell',
  {'entities': [(16, 22, 'control type')]}),
 ('# 4ft emergency sections w/  battery pack 1250 lumens',
  {'entities': [(16, 22, 'control type')]}),
 ('# 4ft emergency sections w/  battery pack 1250 lumens',
  {'entities': [(16, 22, 'control type')]}),
 ('.125 wall thickness', {'entities': [(16, 22, 'control type')]}),
 ('.125 clear polycarbonate lamp side',
  {'entities': [(16, 22, 'control type')]}),
 ('.125 dr prismatic acrylic lamp side',
  {'entities': [(16, 22, 'control type')]}),
 ('.125 prismatic polycarbonate lamp side',
  {'entities': [(16, 22, 'control type')]}),
 ('.156 prismatic polycarbonate lamp side',
  {'entities': [(16, 22, 'control type')]}),
 ('.187 clear tempered glass environmental side',
  {'entities': [(16, 22, 'control type')]})]


import json

# Read output json file from 
with open('input_json.json') as data_file:    
    data = json.load(data_file)

# Extract original sentences
sentences_list = data['_referenced_fss']['12']['sofaString'].split('rn')

# Extract entity start/ end positions and names
ent_loc = data['_views']['_InitialView']['NamedEntity']

# Extract Sentence start/ end positions
Sentence = data['_views']['_InitialView']['Sentence']

# Set first sentence starting position 0
Sentence[0]['begin'] = 0

# Prepare spacy formatted training data
TRAIN_DATA = []
ent_list = []
for sl in range(len(Sentence)):
    ent_list_sen = []
    for el in range(len(ent_loc)):
        if(ent_loc[el]['begin'] >= Sentence[sl]['begin'] and ent_loc[el]['end'] <= Sentence[sl]['end']):
            ## Need to subtract entity location with sentence begining as webanno generate data by treating document as a whole
            ent_list_sen.append([(ent_loc[el]['begin']-Sentence[sl]['begin']),(ent_loc[el]['end']-Sentence[sl]['begin']),ent_loc[el]['value']])
    ent_list.append(ent_list_sen)
    ## Create blank dictionary
    ent_dic = {}
    ## Fill value to the dictionary
    ent_dic['entities'] = ent_list[-1]
    ## Prepare final training data
    TRAIN_DATA.append([sentences_list[sl],ent_dic])
    
TRAIN_DATA

In [ ]:
excelcy = ExcelCy()
# load configuration from XLSX or YML or JSON
# excelcy.load(file_path='test_data_01.xlsx')
# or define manually
excelcy.storage.config = Config(nlp_base='en_core_web_sm', train_iteration=2, train_drop=0.2)
print(json.dumps(excelcy.storage.items(), indent=2))

# add sources
excelcy.storage.source.add(kind='text', value='Robertus Johansyah is the maintainer ExcelCy')
excelcy.storage.source.add(kind='textract', value='tests/data/source/test_source_01.txt')
excelcy.discover()
print(json.dumps(excelcy.storage.items(), indent=2))

# add phrase matcher Robertus Johansyah -> PERSON
excelcy.storage.prepare.add(kind='phrase', value='Robertus Johansyah', entity='PERSON')
excelcy.prepare()
print(json.dumps(excelcy.storage.items(), indent=2))

# train it
excelcy.train()
print(json.dumps(excelcy.storage.items(), indent=2))

# test it
doc = excelcy.nlp('Robertus Johansyah is maintainer ExcelCy')
print(json.dumps(excelcy.storage.items(), indent=2))

NameError: name 'Config' is not defined

In [ ]:
doc = excelcy.nlp('_ pdt occupancy sensor & photocell')
# now it recognise the company name
print([(ent.label_, ent.text) for ent in doc.ents])

[('PERSON', 'pdt occupancy sensor')]


In [ ]:
##create training data
###### Prepare Spacy formatted training data for custom NER #######


  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\~rsly\\msgpack\\_packer.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [61]:

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model) # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en") # create blank Language class
        print("Created blank 'en' model")
        # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner")
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

        # get names of other pipes to disable them during training
        pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
        with nlp.disable_pipes(*other_pipes):  # only train NER
            # reset and initialize the weights randomly – but only if we're
            # training a new model
            if model is None:
                nlp.begin_training()
            for itn in range(n_iter):
                random.shuffle(TRAIN_DATA)
                losses = {}
                # batch up the examples using spaCy's minibatch
                batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
                for batch in batches:
                    for text, annotations in batch:
                        # create Example
                        print(text)
                        doc = nlp.make_doc(text)
                        example = Example.from_dict(doc, annotations)
                        print(example)
                        # Update the model
                        nlp.update(
                            [example],
                            drop=0.5,  # dropout - make it harder to memorise data
                            losses=losses,
                        )
                        print("Losses", losses)
                # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])   

In [62]:
main(None, "model", 30)

Created blank 'en' model
_ pdt occupancy sensors & photocells
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['_', 'pdt', 'occupancy', 'sensors', '&', 'photocells'], 'SPACY': [True, True, True, True, True, False], 'TAG': ['', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', ''], 'POS': ['', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5], 'DEP': ['', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0]}}
Losses {'ner': 3.0}
.187 clear tempered glass environmental side
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['.187', 'clear', 'tempered', 'glass', 'environmental', 'side'], 'SPACY': [True, True, True, True, True, False], 'TAG': ['', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', ''], 'POS': ['', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5], 'DEP': ['', '', '', '

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "_ pdt occupancy sensors & photocells" with entities "[(16, 22, 'control type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text ".187 clear tempered glass environmental side" with entities "[(16, 22, 'control type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entit

Losses {'ner': 7.9636454582214355}
# 4ft emergency sections w/  battery pack 1250 lumens
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['#', '4', 'ft', 'emergency', 'sections', 'w/', ' ', 'battery', 'pack', '1250', 'lumens'], 'SPACY': [True, False, True, True, True, True, False, True, True, True, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', ''], 'POS': ['', '', '', '', '', '', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', '', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'DEP': ['', '', '', '', '', '', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}}
Losses {'ner': 10.380737155675888}
# 4ft emergency sections w/  battery pack 1250 lumens
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': 

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text ".156 prismatic polycarbonate lamp side" with entities "[(16, 22, 'control type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text ".125 dr prismatic acrylic lamp side" with entities "[(16, 22, 'control type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Streaming output truncated to the last 5000 lines.
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['.156', 'prismatic', 'polycarbonate', 'lamp', 'side'], 'SPACY': [True, True, True, True, False], 'TAG': ['', '', '', '', ''], 'LEMMA': ['', '', '', '', ''], 'POS': ['', '', '', '', ''], 'MORPH': ['', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4], 'DEP': ['', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0]}}
Losses {'ner': 3.1715286483657735e-07}
_ pdt occupancy sensor & photocell
{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'U-control type', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['_', 'pdt', 'occupancy', 'sensor', '&', 'photocell'], 'SPACY': [True, True, True, True, True, False], 'TAG': ['', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', ''], 'POS': ['', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5], 'DEP': ['', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0

In [70]:
output_dir = "model"
#Test the saved model
#Pass test data here
test_text = "Top 10 sensor"
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text) 

Loading from model
control type sensor


In [69]:
# test the saved model
output_dir = "model"
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
  doc = nlp2(text)
  print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
  print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])  


Loading from model
Entities []
Tokens [('.187', '', 2), ('clear', '', 2), ('tempered', '', 2), ('glass', '', 2), ('environmental', '', 2), ('side', '', 2)]
Entities []
Tokens [('.125', '', 2), ('prismatic', '', 2), ('polycarbonate', '', 2), ('lamp', '', 2), ('side', '', 2)]
Entities [('sensors', 'control type')]
Tokens [('_', '', 2), ('pdt', '', 2), ('occupancy', '', 2), ('sensors', 'control type', 3), ('&', '', 2), ('photocells', '', 2)]
Entities []
Tokens [('.125', '', 2), ('wall', '', 2), ('thickness', '', 2)]
Entities []
Tokens [('#', '', 2), ('4', '', 2), ('ft', '', 2), ('emergency', '', 2), ('sections', '', 2), ('w/', '', 2), (' ', '', 2), ('battery', '', 2), ('pack', '', 2), ('1250', '', 2), ('lumens', '', 2)]
Entities []
Tokens [('.125', '', 2), ('clear', '', 2), ('polycarbonate', '', 2), ('lamp', '', 2), ('side', '', 2)]
Entities [('sensor', 'control type')]
Tokens [('_', '', 2), ('pdt', '', 2), ('occupancy', '', 2), ('sensor', 'control type', 3), ('&', '', 2), ('photocell', '

In [ ]:

/content/model